# Deep Search and the Common Client

![](./media/AD_Banner.jpg)
<a id="top">

## You only need to run the Magic command %run if you did not execute init_magic from the command line 

In [ ]:
# %run adccl.ipynb #Not required if you have run init_magic

## If you setup the Tell Me Large Language Model interface the below will run for you

In [ ]:
%adccl tell me how would I create a workspace then search for ibuprofen using deepsearch  then display them with a viewer


In [ ]:
%adccl get workspace default

In [ ]:
%adccl remove toolkit DS4SD

In [ ]:
%adccl get status

In [ ]:
%adccl add toolkit DS4SD

In [ ]:
%adccl get status

In [ ]:
%adccl set context DS4SD

## Displaying Domains ##

**Display Domains:**

This is going to query the Deep Search system for the data collections available on the _Materials Science_ domain.
In the list we will find a combination of database and document collections.

Interesting data collections for this examples are PubChem and USTPO pre-processed patents.
Deep Search is regularly parsing the PubChem database to index molecules and their properties.

![displayDomains](data:image/svg+xml,%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20width%3D%22683%22%20height%3D%2237%22%3E%3Cdefs%3E%3Cstyle%20type%3D%22text%2Fcss%22%3E%40namespace%20%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%3B%20.line%20%7Bfill%3A%20none%3B%20stroke%3A%20%23002233%3B%20stroke-width%3A%201%3B%7D%20.bold-line%20%7Bstroke%3A%20%23000E14%3B%20shape-rendering%3A%20crispEdges%3B%20stroke-width%3A%202%3B%7D%20.thin-line%20%7Bstroke%3A%20%2300141F%3B%20shape-rendering%3A%20crispEdges%7D%20.filled%20%7Bfill%3A%20%23002233%3B%20stroke%3A%20none%3B%7D%20text.terminal%20%7Bfont-family%3A%20Verdana%2C%20Sans-serif%3B%20font-size%3A%2012px%3B%20fill%3A%20%23000E14%3B%20font-weight%3A%20bold%3B%20%7D%20text.nonterminal%20%7Bfont-family%3A%20Verdana%2C%20Sans-serif%3B%20font-size%3A%2012px%3B%20fill%3A%20%2300111A%3B%20font-weight%3A%20normal%3B%20%7D%20text.regexp%20%7Bfont-family%3A%20Verdana%2C%20Sans-serif%3B%20font-size%3A%2012px%3B%20fill%3A%20%2300141F%3B%20font-weight%3A%20normal%3B%20%7D%20rect%2C%20circle%2C%20polygon%20%7Bfill%3A%20%23002233%3B%20stroke%3A%20%23002233%3B%7D%20rect.terminal%20%7Bfill%3A%20%234DC3FF%3B%20stroke%3A%20%23002233%3B%20stroke-width%3A%201%3B%7D%20rect.nonterminal%20%7Bfill%3A%20%239EDFFF%3B%20stroke%3A%20%23002233%3B%20stroke-width%3A%201%3B%7D%20rect.text%20%7Bfill%3A%20none%3B%20stroke%3A%20none%3B%7D%20polygon.regexp%20%7Bfill%3A%20%23C7ECFF%3B%20stroke%3A%20%23002233%3B%20stroke-width%3A%201%3B%7D%3C%2Fstyle%3E%3C%2Fdefs%3E%3Cpolygon%20points%3D%229%2017%201%2013%201%2021%22%2F%3E%3Cpolygon%20points%3D%2217%2017%209%2013%209%2021%22%2F%3E%3Crect%20x%3D%2231%22%20y%3D%223%22%20width%3D%2250%22%20height%3D%2232%22%20rx%3D%2210%22%2F%3E%3Crect%20x%3D%2229%22%20y%3D%221%22%20width%3D%2250%22%20height%3D%2232%22%20class%3D%22terminal%22%20rx%3D%2210%22%2F%3E%3Ctext%20class%3D%22terminal%22%20x%3D%2239%22%20y%3D%2221%22%3Eexec%3C%2Ftext%3E%3Crect%20x%3D%22101%22%20y%3D%223%22%20width%3D%22178%22%20height%3D%2232%22%20rx%3D%2210%22%2F%3E%3Crect%20x%3D%2299%22%20y%3D%221%22%20width%3D%22178%22%20height%3D%2232%22%20class%3D%22terminal%22%20rx%3D%2210%22%2F%3E%3Ctext%20class%3D%22terminal%22%20x%3D%22109%22%20y%3D%2221%22%3EDISPLAY_COLLECTION%3C%2Ftext%3E%3Crect%20x%3D%22299%22%20y%3D%223%22%20width%3D%2226%22%20height%3D%2232%22%20rx%3D%2210%22%2F%3E%3Crect%20x%3D%22297%22%20y%3D%221%22%20width%3D%2226%22%20height%3D%2232%22%20class%3D%22terminal%22%20rx%3D%2210%22%2F%3E%3Ctext%20class%3D%22terminal%22%20x%3D%22307%22%20y%3D%2221%22%3E%28%3C%2Ftext%3E%3Crect%20x%3D%22345%22%20y%3D%223%22%20width%3D%2280%22%20height%3D%2232%22%20rx%3D%2210%22%2F%3E%3Crect%20x%3D%22343%22%20y%3D%221%22%20width%3D%2280%22%20height%3D%2232%22%20class%3D%22terminal%22%20rx%3D%2210%22%2F%3E%3Ctext%20class%3D%22terminal%22%20x%3D%22353%22%20y%3D%2221%22%3Edomain%3D%3C%2Ftext%3E%3Crect%20x%3D%22445%22%20y%3D%223%22%20width%3D%22164%22%20height%3D%2232%22%2F%3E%3Crect%20x%3D%22443%22%20y%3D%221%22%20width%3D%22164%22%20height%3D%2232%22%20class%3D%22nonterminal%22%2F%3E%3Ctext%20class%3D%22nonterminal%22%20x%3D%22453%22%20y%3D%2221%22%3Equoted_domain_name%3C%2Ftext%3E%3Crect%20x%3D%22629%22%20y%3D%223%22%20width%3D%2226%22%20height%3D%2232%22%20rx%3D%2210%22%2F%3E%3Crect%20x%3D%22627%22%20y%3D%221%22%20width%3D%2226%22%20height%3D%2232%22%20class%3D%22terminal%22%20rx%3D%2210%22%2F%3E%3Ctext%20class%3D%22terminal%22%20x%3D%22637%22%20y%3D%2221%22%3E%29%3C%2Ftext%3E%3Cpath%20class%3D%22line%22%20d%3D%22m17%2017%20h2%20m0%200%20h10%20m50%200%20h10%20m0%200%20h10%20m178%200%20h10%20m0%200%20h10%20m26%200%20h10%20m0%200%20h10%20m80%200%20h10%20m0%200%20h10%20m164%200%20h10%20m0%200%20h10%20m26%200%20h10%20m3%200%20h-3%22%2F%3E%3Cpolygon%20points%3D%22673%2017%20681%2013%20681%2021%22%2F%3E%3Cpolygon%20points%3D%22673%2017%20665%2013%20665%2021%22%2F%3E%3C%2Fsvg%3E)
```
Example Command:  exec display_collection( domain='Materials Science' ) ```

In [ ]:
%adccl exec display_collection( domain='Materials Science' )

## Searching the Deep Search Repository

The Deep Search command for searching repositories uses the below syntax:

![search](data:image/svg+xml,%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20width%3D%22823%22%20height%3D%22277%22%3E%3Cdefs%3E%3Cstyle%20type%3D%22text%2Fcss%22%3E%40namespace%20%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%3B%20.line%20%7Bfill%3A%20none%3B%20stroke%3A%20%23002233%3B%20stroke-width%3A%201%3B%7D%20.bold-line%20%7Bstroke%3A%20%23000E14%3B%20shape-rendering%3A%20crispEdges%3B%20stroke-width%3A%202%3B%7D%20.thin-line%20%7Bstroke%3A%20%2300141F%3B%20shape-rendering%3A%20crispEdges%7D%20.filled%20%7Bfill%3A%20%23002233%3B%20stroke%3A%20none%3B%7D%20text.terminal%20%7Bfont-family%3A%20Verdana%2C%20Sans-serif%3B%20font-size%3A%2012px%3B%20fill%3A%20%23000E14%3B%20font-weight%3A%20bold%3B%20%7D%20text.nonterminal%20%7Bfont-family%3A%20Verdana%2C%20Sans-serif%3B%20font-size%3A%2012px%3B%20fill%3A%20%2300111A%3B%20font-weight%3A%20normal%3B%20%7D%20text.regexp%20%7Bfont-family%3A%20Verdana%2C%20Sans-serif%3B%20font-size%3A%2012px%3B%20fill%3A%20%2300141F%3B%20font-weight%3A%20normal%3B%20%7D%20rect%2C%20circle%2C%20polygon%20%7Bfill%3A%20%23002233%3B%20stroke%3A%20%23002233%3B%7D%20rect.terminal%20%7Bfill%3A%20%234DC3FF%3B%20stroke%3A%20%23002233%3B%20stroke-width%3A%201%3B%7D%20rect.nonterminal%20%7Bfill%3A%20%239EDFFF%3B%20stroke%3A%20%23002233%3B%20stroke-width%3A%201%3B%7D%20rect.text%20%7Bfill%3A%20none%3B%20stroke%3A%20none%3B%7D%20polygon.regexp%20%7Bfill%3A%20%23C7ECFF%3B%20stroke%3A%20%23002233%3B%20stroke-width%3A%201%3B%7D%3C%2Fstyle%3E%3C%2Fdefs%3E%3Cpolygon%20points%3D%229%2017%201%2013%201%2021%22%2F%3E%3Cpolygon%20points%3D%2217%2017%209%2013%209%2021%22%2F%3E%3Crect%20x%3D%2231%22%20y%3D%223%22%20width%3D%2264%22%20height%3D%2232%22%20rx%3D%2210%22%2F%3E%3Crect%20x%3D%2229%22%20y%3D%221%22%20width%3D%2264%22%20height%3D%2232%22%20class%3D%22terminal%22%20rx%3D%2210%22%2F%3E%3Ctext%20class%3D%22terminal%22%20x%3D%2239%22%20y%3D%2221%22%3Esearch%3C%2Ftext%3E%3Crect%20x%3D%22115%22%20y%3D%223%22%20width%3D%2248%22%20height%3D%2232%22%20rx%3D%2210%22%2F%3E%3Crect%20x%3D%22113%22%20y%3D%221%22%20width%3D%2248%22%20height%3D%2232%22%20class%3D%22terminal%22%20rx%3D%2210%22%2F%3E%3Ctext%20class%3D%22terminal%22%20x%3D%22123%22%20y%3D%2221%22%3EFOR%3C%2Ftext%3E%3Crect%20x%3D%22183%22%20y%3D%223%22%20width%3D%22160%22%20height%3D%2232%22%2F%3E%3Crect%20x%3D%22181%22%20y%3D%221%22%20width%3D%22160%22%20height%3D%2232%22%20class%3D%22nonterminal%22%2F%3E%3Ctext%20class%3D%22nonterminal%22%20x%3D%22191%22%20y%3D%2221%22%3Equoted_search_string%3C%2Ftext%3E%3Crect%20x%3D%22363%22%20y%3D%223%22%20width%3D%2264%22%20height%3D%2232%22%20rx%3D%2210%22%2F%3E%3Crect%20x%3D%22361%22%20y%3D%221%22%20width%3D%2264%22%20height%3D%2232%22%20class%3D%22terminal%22%20rx%3D%2210%22%2F%3E%3Ctext%20class%3D%22terminal%22%20x%3D%22371%22%20y%3D%2221%22%3EUSING%3C%2Ftext%3E%3Crect%20x%3D%22447%22%20y%3D%223%22%20width%3D%2226%22%20height%3D%2232%22%20rx%3D%2210%22%2F%3E%3Crect%20x%3D%22445%22%20y%3D%221%22%20width%3D%2226%22%20height%3D%2232%22%20class%3D%22terminal%22%20rx%3D%2210%22%2F%3E%3Ctext%20class%3D%22terminal%22%20x%3D%22455%22%20y%3D%2221%22%3E%28%3C%2Ftext%3E%3Crect%20x%3D%22493%22%20y%3D%223%22%20width%3D%22106%22%20height%3D%2232%22%20rx%3D%2210%22%2F%3E%3Crect%20x%3D%22491%22%20y%3D%221%22%20width%3D%22106%22%20height%3D%2232%22%20class%3D%22terminal%22%20rx%3D%2210%22%2F%3E%3Ctext%20class%3D%22terminal%22%20x%3D%22501%22%20y%3D%2221%22%3Eindex_type%3D%3C%2Ftext%3E%3Crect%20x%3D%22619%22%20y%3D%223%22%20width%3D%22150%22%20height%3D%2232%22%2F%3E%3Crect%20x%3D%22617%22%20y%3D%221%22%20width%3D%22150%22%20height%3D%2232%22%20class%3D%22nonterminal%22%2F%3E%3Ctext%20class%3D%22nonterminal%22%20x%3D%22627%22%20y%3D%2221%22%3Equoted_index_name%3C%2Ftext%3E%3Crect%20x%3D%2245%22%20y%3D%2269%22%20width%3D%2296%22%20height%3D%2232%22%20rx%3D%2210%22%2F%3E%3Crect%20x%3D%2243%22%20y%3D%2267%22%20width%3D%2296%22%20height%3D%2232%22%20class%3D%22terminal%22%20rx%3D%2210%22%2F%3E%3Ctext%20class%3D%22terminal%22%20x%3D%2253%22%20y%3D%2287%22%3Eelastic_id%3D%3C%2Ftext%3E%3Crect%20x%3D%22161%22%20y%3D%2269%22%20width%3D%2256%22%20height%3D%2232%22%2F%3E%3Crect%20x%3D%22159%22%20y%3D%2267%22%20width%3D%2256%22%20height%3D%2232%22%20class%3D%22nonterminal%22%2F%3E%3Ctext%20class%3D%22nonterminal%22%20x%3D%22169%22%20y%3D%2287%22%3Estring%3C%2Ftext%3E%3Crect%20x%3D%2265%22%20y%3D%22113%22%20width%3D%2298%22%20height%3D%2232%22%20rx%3D%2210%22%2F%3E%3Crect%20x%3D%2263%22%20y%3D%22111%22%20width%3D%2298%22%20height%3D%2232%22%20class%3D%22terminal%22%20rx%3D%2210%22%2F%3E%3Ctext%20class%3D%22terminal%22%20x%3D%2273%22%20y%3D%22131%22%3Epage_size%3D%3C%2Ftext%3E%3Crect%20x%3D%2265%22%20y%3D%22157%22%20width%3D%22122%22%20height%3D%2232%22%20rx%3D%2210%22%2F%3E%3Crect%20x%3D%2263%22%20y%3D%22155%22%20width%3D%22122%22%20height%3D%2232%22%20class%3D%22terminal%22%20rx%3D%2210%22%2F%3E%3Ctext%20class%3D%22terminal%22%20x%3D%2273%22%20y%3D%22175%22%3Eedit_distance%3D%3C%2Ftext%3E%3Crect%20x%3D%22227%22%20y%3D%22113%22%20width%3D%2264%22%20height%3D%2232%22%2F%3E%3Crect%20x%3D%22225%22%20y%3D%22111%22%20width%3D%2264%22%20height%3D%2232%22%20class%3D%22nonterminal%22%2F%3E%3Ctext%20class%3D%22nonterminal%22%20x%3D%22235%22%20y%3D%22131%22%3Einteger%3C%2Ftext%3E%3Crect%20x%3D%22331%22%20y%3D%2269%22%20width%3D%2226%22%20height%3D%2232%22%20rx%3D%2210%22%2F%3E%3Crect%20x%3D%22329%22%20y%3D%2267%22%20width%3D%2226%22%20height%3D%2232%22%20class%3D%22terminal%22%20rx%3D%2210%22%2F%3E%3Ctext%20class%3D%22terminal%22%20x%3D%22339%22%20y%3D%2287%22%3E%29%3C%2Ftext%3E%3Crect%20x%3D%22377%22%20y%3D%2269%22%20width%3D%2270%22%20height%3D%2232%22%20rx%3D%2210%22%2F%3E%3Crect%20x%3D%22375%22%20y%3D%2267%22%20width%3D%2270%22%20height%3D%2232%22%20class%3D%22terminal%22%20rx%3D%2210%22%2F%3E%3Ctext%20class%3D%22terminal%22%20x%3D%22385%22%20y%3D%2287%22%3ESHOW%28%3C%2Ftext%3E%3Crect%20x%3D%22487%22%20y%3D%2269%22%20width%3D%2258%22%20height%3D%2232%22%20rx%3D%2210%22%2F%3E%3Crect%20x%3D%22485%22%20y%3D%2267%22%20width%3D%2258%22%20height%3D%2232%22%20class%3D%22terminal%22%20rx%3D%2210%22%2F%3E%3Ctext%20class%3D%22terminal%22%20x%3D%22495%22%20y%3D%2287%22%3EDOCS%3C%2Ftext%3E%3Crect%20x%3D%22487%22%20y%3D%22113%22%20width%3D%2256%22%20height%3D%2232%22%20rx%3D%2210%22%2F%3E%3Crect%20x%3D%22485%22%20y%3D%22111%22%20width%3D%2256%22%20height%3D%2232%22%20class%3D%22terminal%22%20rx%3D%2210%22%2F%3E%3Ctext%20class%3D%22terminal%22%20x%3D%22495%22%20y%3D%22131%22%3EDATA%3C%2Ftext%3E%3Crect%20x%3D%22585%22%20y%3D%2269%22%20width%3D%2226%22%20height%3D%2232%22%20rx%3D%2210%22%2F%3E%3Crect%20x%3D%22583%22%20y%3D%2267%22%20width%3D%2226%22%20height%3D%2232%22%20class%3D%22terminal%22%20rx%3D%2210%22%2F%3E%3Ctext%20class%3D%22terminal%22%20x%3D%22593%22%20y%3D%2287%22%3E%29%3C%2Ftext%3E%3Crect%20x%3D%22651%22%20y%3D%22101%22%20width%3D%22130%22%20height%3D%2232%22%20rx%3D%2210%22%2F%3E%3Crect%20x%3D%22649%22%20y%3D%2299%22%20width%3D%22130%22%20height%3D%2232%22%20class%3D%22terminal%22%20rx%3D%2210%22%2F%3E%3Ctext%20class%3D%22terminal%22%20x%3D%22659%22%20y%3D%22119%22%3EESTIMATE%20ONLY%3C%2Ftext%3E%3Crect%20x%3D%22363%22%20y%3D%22243%22%20width%3D%22118%22%20height%3D%2232%22%20rx%3D%2210%22%2F%3E%3Crect%20x%3D%22361%22%20y%3D%22241%22%20width%3D%22118%22%20height%3D%2232%22%20class%3D%22terminal%22%20rx%3D%2210%22%2F%3E%3Ctext%20class%3D%22terminal%22%20x%3D%22371%22%20y%3D%22261%22%3EReturn%20as%20data%3C%2Ftext%3E%3Crect%20x%3D%22541%22%20y%3D%22243%22%20width%3D%2278%22%20height%3D%2232%22%20rx%3D%2210%22%2F%3E%3Crect%20x%3D%22539%22%20y%3D%22241%22%20width%3D%2278%22%20height%3D%2232%22%20class%3D%22terminal%22%20rx%3D%2210%22%2F%3E%3Ctext%20class%3D%22terminal%22%20x%3D%22549%22%20y%3D%22261%22%3ESAVE%20AS%3C%2Ftext%3E%3Crect%20x%3D%22639%22%20y%3D%22243%22%20width%3D%22136%22%20height%3D%2232%22%2F%3E%3Crect%20x%3D%22637%22%20y%3D%22241%22%20width%3D%22136%22%20height%3D%2232%22%20class%3D%22nonterminal%22%2F%3E%3Ctext%20class%3D%22nonterminal%22%20x%3D%22647%22%20y%3D%22261%22%3Equoted_csv_name%3C%2Ftext%3E%3Cpath%20class%3D%22line%22%20d%3D%22m17%2017%20h2%20m0%200%20h10%20m64%200%20h10%20m0%200%20h10%20m48%200%20h10%20m0%200%20h10%20m160%200%20h10%20m0%200%20h10%20m64%200%20h10%20m0%200%20h10%20m26%200%20h10%20m0%200%20h10%20m106%200%20h10%20m0%200%20h10%20m150%200%20h10%20m2%200%20l2%200%20m2%200%20l2%200%20m2%200%20l2%200%20m-788%2066%20l2%200%20m2%200%20l2%200%20m2%200%20l2%200%20m22%200%20h10%20m96%200%20h10%20m0%200%20h10%20m56%200%20h10%20m0%200%20h74%20m-286%200%20h20%20m266%200%20h20%20m-306%200%20q10%200%2010%2010%20m286%200%20q0%20-10%2010%20-10%20m-296%2010%20v24%20m286%200%20v-24%20m-286%2024%20q0%2010%2010%2010%20m266%200%20q10%200%2010%20-10%20m-256%2010%20h10%20m98%200%20h10%20m0%200%20h24%20m-162%200%20h20%20m142%200%20h20%20m-182%200%20q10%200%2010%2010%20m162%200%20q0%20-10%2010%20-10%20m-172%2010%20v24%20m162%200%20v-24%20m-162%2024%20q0%2010%2010%2010%20m142%200%20q10%200%2010%20-10%20m-152%2010%20h10%20m122%200%20h10%20m20%20-44%20h10%20m64%200%20h10%20m20%20-44%20h10%20m26%200%20h10%20m0%200%20h10%20m70%200%20h10%20m20%200%20h10%20m58%200%20h10%20m-98%200%20h20%20m78%200%20h20%20m-118%200%20q10%200%2010%2010%20m98%200%20q0%20-10%2010%20-10%20m-108%2010%20v24%20m98%200%20v-24%20m-98%2024%20q0%2010%2010%2010%20m78%200%20q10%200%2010%20-10%20m-88%2010%20h10%20m56%200%20h10%20m0%200%20h2%20m20%20-44%20h10%20m26%200%20h10%20m20%200%20h10%20m0%200%20h140%20m-170%200%20h20%20m150%200%20h20%20m-190%200%20q10%200%2010%2010%20m170%200%20q0%20-10%2010%20-10%20m-180%2010%20v12%20m170%200%20v-12%20m-170%2012%20q0%2010%2010%2010%20m150%200%20q10%200%2010%20-10%20m-160%2010%20h10%20m130%200%20h10%20m22%20-32%20l2%200%20m2%200%20l2%200%20m2%200%20l2%200%20m-502%20142%20l2%200%20m2%200%20l2%200%20m2%200%20l2%200%20m22%200%20h10%20m0%200%20h128%20m-158%200%20h20%20m138%200%20h20%20m-178%200%20q10%200%2010%2010%20m158%200%20q0%20-10%2010%20-10%20m-168%2010%20v12%20m158%200%20v-12%20m-158%2012%20q0%2010%2010%2010%20m138%200%20q10%200%2010%20-10%20m-148%2010%20h10%20m118%200%20h10%20m40%20-32%20h10%20m0%200%20h244%20m-274%200%20h20%20m254%200%20h20%20m-294%200%20q10%200%2010%2010%20m274%200%20q0%20-10%2010%20-10%20m-284%2010%20v12%20m274%200%20v-12%20m-274%2012%20q0%2010%2010%2010%20m254%200%20q10%200%2010%20-10%20m-264%2010%20h10%20m78%200%20h10%20m0%200%20h10%20m136%200%20h10%20m23%20-32%20h-3%22%2F%3E%3Cpolygon%20points%3D%22813%20225%20821%20221%20821%20229%22%2F%3E%3Cpolygon%20points%3D%22813%20225%20805%20221%20805%20229%22%2F%3E%3C%2Fsvg%3E)

**For Example:** search for 'ide("power conversion efficiency" OR PCE) AND organ* ' using (index_key='arxiv' elastic_id=default ) show (docs) estimate only

### **Searching for Documents on a Subject**



In this section we search for documents in the arXiv.org data collection matching the input query. For each matched document we return the title, authors as well as the link to the original document on arXix.org

***This example will demonstrate***

1. How to address a specific data collection
2. How to choose which component of the documents should be returned
3. How to iterate through the complete data collection by fetching page_size=50 results at the time

First we will run and get an estimate of how many documents may appear in the search so we know we are pulling back a manageable amount

In [ ]:
%adccl search for 'ide("power conversion efficiency" OR PCE) AND organ* ' using \
(index_key='arxiv' elastic_id=default ) show (docs) estimate only

### Retrieving results ###
Now we will pull back all results, if we do not use the `return as data` clause the results will be returned as pandas styler object.

In [ ]:
df_styler= %adccl search for 'ide("power conversion efficiency" OR PCE) AND organ* ' using \
(index_key='arxiv' elastic_id=default edit_distance=20 ) show (data docs) 

The Styler object can be viewed straight away or assigned as a variable, to extract the raw data we would reference `df_styler.data` to return the base data in a data frame.
make sure the cell you are referencing has scrolling enalbled by setting the properties for the cell, this way you can scroll through all the data, a window rather than taking up the entire notebook.

In [ ]:
df_styler

### Simply viewing the Results ###
here we will run the search straight from the magic command view it straight away

In [ ]:
%adccl search for '" carbon capture membrane" ' using (index_key='arxiv' elastic_id=default  ) show (data docs)  

## **Search _Ibuprofen_ on PubChem** 

In this section we search for all PubChem entries which contain the string _Ibuprofen_.

In the results table we see the name of the chemical, its molecule SMILES and some properties such as the molecular weight and the solubility.


In [ ]:
%adccl SEARCH FOR  'Ibuprofen' USING (index_key='pubchem')   SHOW (data) 

### Working with the results as Data ###
By using the `return as data` clause we will return the data as a raw data frame , this way we will be able to pass it to other utilities.

In [ ]:
smiles_molecules = %adccl search for 'identifiers._name:"smiles#ccc(coc(=o)cs)(c(=o)c(=o)cs)c(=o)c(=o)cs" ' using (index_key='patent-uspto-smiles') show (data) return as data

In [ ]:
display(smiles_molecules)

### Displaying the molecules in a viewer ###
Now we will view the molecules and select to subset the moleculess for further viewing.

In [ ]:

%adccl show molecules using dataframe smiles_molecules

## **Now We display via mols2grid object and do selections to a data frame** 



In [ ]:
x = %adccl show molecules using dataframe smiles_molecules as molsobject

In [ ]:
x.display(**{'subset': ['NAME']})

Select an option then run the next celll

In [ ]:
x.get_selection()